# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="./Tennis_Windows_x86_64/Tennis.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [5]:
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

Score (max over agents) from episode 1: 0.0
Score (max over agents) from episode 2: 0.0
Score (max over agents) from episode 3: 0.0
Score (max over agents) from episode 4: 0.0
Score (max over agents) from episode 5: 0.0


When finished, you can close the environment.

In [6]:
#env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

References:
* [MADDPG paper](https://arxiv.org/pdf/1706.02275.pdf)
* [Blog by Amit Patel](https://medium.com/@amitpatel.gt/maddpg-91caa221d75e) and associated code on [GitHib](https://github.com/gtg162y/DRLND/blob/master/P3_Collab_Compete/Tennis_Udacity_Workspace.ipynb)

In [7]:
from datetime import datetime
import math

#temporary
#import numpy as np
import torch
import torch.optim as optim
import torch.nn.functional as F
from collections import deque, namedtuple
import copy
import random
import pickle
from models import *
import os

In [8]:
from models import *
#this contains actor_network and critic_network classes

from agent import replay_buffer, add_noise
#contains agent and replay buffer classes. Also a gaussian noise generation function

class agent:
    def __init__(self, num_agents, observation_size, action_size,
                 actor_hidden1_size, actor_hidden2_size,
                 critic_hidden1_size, critic_hidden2_size,
                 noise_start,
                 replay_buffer_size, replay_batch_size,
                 gamma, critic_LR, actor_LR, tau):
        
        self.num_agents = num_agents
        self.action_size = action_size
        self.observation_size = observation_size
        
        self.gamma = gamma
        self.tau = tau
        
        self.actors_local = []
        self.critics_local = []
        self.actor_optims = []
        self.critic_optims = []
        for i in range(self.num_agents):
            self.actors_local.append(actor_network(observation_size, actor_hidden1_size, actor_hidden2_size, action_size))
            self.critics_local.append(critic_network(observation_size, critic_hidden1_size, critic_hidden2_size, action_size, self.num_agents))
            self.actor_optims.append(optim.Adam(self.actors_local[i].parameters(), lr=actor_LR))
            self.critic_optims.append(optim.Adam(self.critics_local[i].parameters(), lr=critic_LR))
        
        self.actors_target = copy.deepcopy(self.actors_local)
        self.critics_target = copy.deepcopy(self.critics_local)
        
        self.noise_magnitude = noise_start
        self.step_count = 0
        
        self.replay_batch_size = replay_batch_size
        self.replay_buffer = replay_buffer(replay_buffer_size, replay_batch_size)
        

    
    def get_actions(self, states, noise=True):
        '''
        Given array of all observations (states) for all agents (n_agents,state_size), return actions using each agent's policy
        '''
        actions = []
        for i in range(self.num_agents):
            state = states[i,:]
            self.actors_local[i].eval()
            state = torch.from_numpy(state).float()
            with torch.no_grad():
                action = self.actors_local[i](state).detach().numpy()
            self.actors_local[i].train()
            actions.append(action)
        
        actions = np.array(actions) 
        #shape(n_agents,action_size)
        
        if noise:
            actions = add_noise(actions,self.noise_magnitude)
            
        return(np.array(actions))
    
    def set_noise(self,magnitude):
    #this is for resetting the noise magnitude for whatever reason
        self.noise_magnitude = magnitude
        

    def step(self, states, actions, rewards, next_states, dones, num_updates):
        self.replay_buffer.add(states, actions, rewards, next_states, dones)
        if len(self.replay_buffer) >= self.replay_batch_size:
            for _ in range(num_updates):
                for i in range(self.num_agents):
                    e = self.replay_buffer.sample()
                    self.learn(e, i)
                
    def learn(self, experience, agent_no):
        states, actions, rewards, next_states, dones = experience
        #states and next states: (batch_size, n_agents, observation_size)
        #actions:                (batch_size, n_agents, action_size)
        #rewards and dones       (batch_size, n_agents)

        next_actions = []
        for k in range(self.num_agents):
            local_obs = next_states[:,k,:]
            #local_obs = torch.from_numpy(local_obs).float()
            na = self.actors_target[k](local_obs)
            #size (batch_size,action_size)
            next_actions.append(na)
            
        #update local critic
        states_flat      = states.view(self.replay_batch_size,self.num_agents*self.observation_size)
        next_states_flat = next_states.view(self.replay_batch_size,self.num_agents*self.observation_size)
        actions_flat = actions.view(self.replay_batch_size,self.num_agents*self.action_size)
        
        
        Q_vals = self.critics_local[agent_no](states_flat,actions_flat) #size (batch_size, action_size)
        
        next_actions = torch.stack(next_actions,dim=1).view(self.replay_batch_size,self.num_agents*self.action_size)
        
        Q_next = self.critics_target[agent_no](next_states_flat, next_actions) #size (batch_size, action_size)
        Q_target = rewards[:,agent_no].view(self.replay_batch_size,1) + self.gamma * Q_next * (1-dones[:,agent_no].view(self.replay_batch_size,1))
        
        critic_loss = F.mse_loss(Q_vals,Q_target)
        self.critic_optims[agent_no].zero_grad
        critic_loss.backward()
        self.critic_optims[agent_no].step()

        #update local actor
        actions_to_critic = actions #actions of every agent
        actions_to_critic[:,agent_no,:] = self.actors_local[agent_no](states[:,agent_no,:]) #replace values for this agent with value from policy
        actions_to_critic = actions_to_critic.view(self.replay_batch_size,self.num_agents*self.action_size)

        actor_loss = -self.critics_local[agent_no](states_flat,actions_to_critic).mean()
        self.actor_optims[agent_no].zero_grad()
        actor_loss.backward()
        self.actor_optims[agent_no].step()
        
        #soft update target networks
        self.actors_target[agent_no] = self.soft_update(self.actors_local[agent_no], self.actors_target[agent_no], self.tau)
        self.critics_target[agent_no] = self.soft_update(self.critics_local[agent_no], self.critics_target[agent_no], self.tau)
        
        self.step_count +=1
        
    def soft_update(self,local,target,tau):
        for name, param in target.state_dict().items():
            l, t = local.state_dict()[name], target.state_dict()[name] #local,target
            new = l * tau + t * (1.0-tau)
            target.state_dict()[name].copy_(new)
        return(target)
    
    def save_checkpoints(self, root, run_name, episode_number, scores=None):
        '''
        In the directory root, creates file run_name if it doesn't exist already. Saves all the model weights (state_dict) 
        for all the actor and critic local networks (i.e. one of each per agent).
        Optional scores argument will also save a scores file in same location as a pickle object
        '''
        
        #create sub-folder for this run if it doesn't exist already
        if not(os.path.exists(root+'/'+run_name)):
            os.mkdir(root+'/'+run_name)
        
        for i in range(self.num_agents):
            torch.save(self.actors_local[i].state_dict(), root+'/'+run_name+'/actor-agent_%s-episode_%s.pth'%(i,episode_number))
            torch.save(self.critics_local[i].state_dict(), root+'/'+run_name+'/critic-agent_%s-episode_%s.pth'%(i,episode_number))
        
        if scores!=None:
            with open(root+'/'+run_name+'/scores-episode_'+episode_number+'.pkl', 'wb') as f:
                pickle.dump(scores,f)
    
    
    def load_checkpoints(self, root, run_name, episode_number, scores_bool=False):
        for i in range(self.num_agents):
            self.actors_local[i].load_state_dict(torch.load(root+'/'+run_name+'/actor-agent_%s-episode_%s.pth'%(i,episode_number)))
            self.critics_local[i].load_state_dict(torch.load(root+'/'+run_name+'/critic-agent_%s-episode_%s.pth'%(i,episode_number)))
        if scores_bool:
            scores = pickle.load(open(root+'/'+run_name+'/scores-episode_'+episode_number+'.pkl','rb'))
            return scores

In [9]:
HIDDEN_SIZE = 300

NOISE_START = 1.5
NOISE_DECAY = 0.998
NOISE_END = 0.05
print('Takes %s decays for noise to reach %s'% (math.log((NOISE_END/NOISE_START),NOISE_DECAY), NOISE_END) )

REPLAY_MAX_SIZE = 1e5
BATCH_SIZE =256

GAMMA=0.99
CRITIC_LR = 1e-4
ACTOR_LR = 1e-4
TAU = 1e-3

N_RANDOM = 300
UPDATES_PER_STEP = 3

checkpoint_dir = "./checkpoints"
run_name = datetime.now().strftime('%Y%m%d_%H%M')
print('Run name = ',run_name)
SAVE_EVERY = 50

NUM_EPISODES = 2500

Agent = agent(num_agents,state_size,action_size,
              HIDDEN_SIZE,HIDDEN_SIZE,HIDDEN_SIZE,HIDDEN_SIZE,
              NOISE_START,
              REPLAY_MAX_SIZE,BATCH_SIZE,
              GAMMA, CRITIC_LR, ACTOR_LR, TAU)

Takes 1698.8975247064295 decays for noise to reach 0.05
Run name =  20200515_1532


In [10]:
def timedelta_to_str(d):
    '''function to format a timedelta string nicely'''
    s = d.seconds
    h = math.floor(s/3600)
    m = math.floor((s-h*3600)/60)
    return('%sh %sm'%(h,m))

In [ ]:
score_history = []

action_history = []

start_time = datetime.now()
for episode in range(NUM_EPISODES):                        
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    #shape (n_agents,observation_size)
    
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    
    if episode<N_RANDOM:
        Agent.set_noise(NOISE_START)
    else:
        Agent.set_noise(max(NOISE_START * NOISE_DECAY**(episode-N_RANDOM), NOISE_END))
    while True:
        if episode<=N_RANDOM:
            actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        else:
            actions = Agent.get_actions(states,noise=True) # all actions between -1 and 1
        
        actions = np.clip(actions, -1, 1)
        action_history.append(actions)
        
        
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        #list of length n_agents
        dones = env_info.local_done                        # see if episode finished
        #list of length n_agents
        
        Agent.step(states, actions, rewards, next_states, dones, num_updates=UPDATES_PER_STEP)
        
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step

        if np.any(dones):                                  # exit loop if episode finished
            break
    
    score_history.append(np.max(scores))
    
    now_time = datetime.now()
    run_time = now_time-start_time
    forecast_time = (run_time / (episode + 1)) * (NUM_EPISODES - episode - 1)
    
    print('episode = %s, score = %.3f, running for: %s, time to go: %s'%(episode+1,np.max(scores),
                                                                    timedelta_to_str(run_time),
                                                                    timedelta_to_str(forecast_time)))

    if (episode%SAVE_EVERY) == 0 & (episode!=0):
        Agent.save_checkpoints(checkpoint_dir,run_name,str(episode), scores=score_history)

Agent.save_checkpoints(checkpoint_dir,run_name,'final', scores=score_history)

episode = 1, score = 0.100, running for: 0h 0m, time to go: 2h 13m
episode = 2, score = 0.000, running for: 0h 0m, time to go: 1h 38m
episode = 3, score = 0.000, running for: 0h 0m, time to go: 1h 25m
episode = 4, score = 0.000, running for: 0h 0m, time to go: 1h 18m
episode = 5, score = 0.000, running for: 0h 0m, time to go: 1h 15m
episode = 6, score = 0.000, running for: 0h 0m, time to go: 1h 12m
episode = 7, score = 0.000, running for: 0h 0m, time to go: 1h 10m
episode = 8, score = 0.000, running for: 0h 0m, time to go: 1h 10m
episode = 9, score = 0.000, running for: 0h 0m, time to go: 1h 9m
episode = 10, score = 0.000, running for: 0h 0m, time to go: 1h 8m
episode = 11, score = 0.090, running for: 0h 0m, time to go: 1h 13m
episode = 12, score = 0.000, running for: 0h 0m, time to go: 1h 12m
episode = 13, score = 0.000, running for: 0h 0m, time to go: 1h 11m
episode = 14, score = 0.100, running for: 0h 0m, time to go: 1h 15m
episode = 15, score = 0.000, running for: 0h 0m, time to go

episode = 122, score = 0.090, running for: 0h 10m, time to go: 3h 30m
episode = 123, score = 0.000, running for: 0h 10m, time to go: 3h 30m
episode = 124, score = 0.000, running for: 0h 10m, time to go: 3h 30m
episode = 125, score = 0.000, running for: 0h 11m, time to go: 3h 29m
episode = 126, score = 0.090, running for: 0h 11m, time to go: 3h 31m
episode = 127, score = 0.100, running for: 0h 11m, time to go: 3h 35m
episode = 128, score = 0.000, running for: 0h 11m, time to go: 3h 34m
episode = 129, score = 0.100, running for: 0h 11m, time to go: 3h 35m
episode = 130, score = 0.000, running for: 0h 11m, time to go: 3h 35m
episode = 131, score = 0.000, running for: 0h 11m, time to go: 3h 35m
episode = 132, score = 0.100, running for: 0h 12m, time to go: 3h 36m
episode = 133, score = 0.000, running for: 0h 12m, time to go: 3h 36m
episode = 134, score = 0.000, running for: 0h 12m, time to go: 3h 35m
episode = 135, score = 0.100, running for: 0h 12m, time to go: 3h 37m
episode = 136, score

episode = 240, score = 0.000, running for: 0h 22m, time to go: 3h 35m
episode = 241, score = 0.100, running for: 0h 23m, time to go: 3h 36m
episode = 242, score = 0.000, running for: 0h 23m, time to go: 3h 36m
episode = 243, score = 0.100, running for: 0h 23m, time to go: 3h 38m
episode = 244, score = 0.000, running for: 0h 23m, time to go: 3h 37m
episode = 245, score = 0.100, running for: 0h 23m, time to go: 3h 38m
episode = 246, score = 0.100, running for: 0h 23m, time to go: 3h 39m
episode = 247, score = 0.000, running for: 0h 24m, time to go: 3h 38m
episode = 248, score = 0.000, running for: 0h 24m, time to go: 3h 38m
episode = 249, score = 0.000, running for: 0h 24m, time to go: 3h 38m
episode = 250, score = 0.000, running for: 0h 24m, time to go: 3h 38m
episode = 251, score = 0.000, running for: 0h 24m, time to go: 3h 38m
episode = 252, score = 0.100, running for: 0h 24m, time to go: 3h 40m
episode = 253, score = 0.000, running for: 0h 24m, time to go: 3h 40m
episode = 254, score

episode = 358, score = 0.000, running for: 0h 34m, time to go: 3h 28m
episode = 359, score = 0.000, running for: 0h 34m, time to go: 3h 28m
episode = 360, score = 0.000, running for: 0h 35m, time to go: 3h 28m
episode = 361, score = 0.000, running for: 0h 35m, time to go: 3h 28m
episode = 362, score = 0.000, running for: 0h 35m, time to go: 3h 28m
episode = 363, score = 0.000, running for: 0h 35m, time to go: 3h 27m
episode = 364, score = 0.100, running for: 0h 35m, time to go: 3h 29m
episode = 365, score = 0.000, running for: 0h 35m, time to go: 3h 28m
episode = 366, score = 0.000, running for: 0h 35m, time to go: 3h 28m
episode = 367, score = 0.000, running for: 0h 35m, time to go: 3h 28m
episode = 368, score = 0.000, running for: 0h 35m, time to go: 3h 28m
episode = 369, score = 0.000, running for: 0h 36m, time to go: 3h 27m
episode = 370, score = 0.000, running for: 0h 36m, time to go: 3h 27m
episode = 371, score = 0.000, running for: 0h 36m, time to go: 3h 27m
episode = 372, score

episode = 476, score = 0.000, running for: 0h 45m, time to go: 3h 14m
episode = 477, score = 0.000, running for: 0h 45m, time to go: 3h 14m
episode = 478, score = 0.000, running for: 0h 45m, time to go: 3h 13m
episode = 479, score = 0.000, running for: 0h 45m, time to go: 3h 13m
episode = 480, score = 0.000, running for: 0h 45m, time to go: 3h 13m
episode = 481, score = 0.000, running for: 0h 46m, time to go: 3h 13m
episode = 482, score = 0.000, running for: 0h 46m, time to go: 3h 13m
episode = 483, score = 0.000, running for: 0h 46m, time to go: 3h 13m
episode = 484, score = 0.000, running for: 0h 46m, time to go: 3h 12m
episode = 485, score = 0.000, running for: 0h 46m, time to go: 3h 12m
episode = 486, score = 0.000, running for: 0h 46m, time to go: 3h 12m
episode = 487, score = 0.000, running for: 0h 46m, time to go: 3h 12m
episode = 488, score = 0.000, running for: 0h 46m, time to go: 3h 12m
episode = 489, score = 0.000, running for: 0h 46m, time to go: 3h 12m
episode = 490, score

In [ ]:
def moving_average(a, n) :
    '''
    Calculates the moving average of an input list a, over a window of size n. 
    Returns the array of averages and the position of the final value in the window for each value
    '''
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    ave = ret[n - 1:] / n
    inds = list(range(n,len(ave)+n))
    return(ave,inds)

In [ ]:
import matplotlib.pyplot as plt

smooth_period = 5
colour='r'

plt.plot(score_history,color=colour,alpha=0.5)
ave, inds = moving_average(score_history,smooth_period)
#plt.plot(inds,ave,color=colour)
plt.show()

In [ ]:
#%matplotlib notebook
plt.plot(noise_history)
plt.show()

In [ ]:
#load pretrained model and get scores history
saved_run_name = run_name #modify this to be whatever run_name was used
ep_no = 'final' #Can choose another episode number
scores = Agent.load_checkpoints(checkpoint_dir,saved_run_name,ep_no,scores_bool=True)

In [ ]:
action_history = np.array(action_history) #step_no, agent, action_size
segment_size = 80 #number of steps in a segment
agent_number = 0
action_number = 0

In [ ]:
print(action_history.shape)

In [ ]:
fig,ax = plt.subplots()
fig2,ax2 = plt.subplots()
seg = 0
bins = np.arange(-1,1.2,0.2)
centres = bins[:-1] + (bins[1:]-bins[:-1])/2



while seg+segment_size <action_history.shape[0]:
    vals = action_history[seg:seg+segment_size,agent_number,action_number]
    n,_,_ = ax.hist(vals,bins=bins)
    ax2.plot(centres,n,label='%s'%(seg))
    ax2.legend()
    print(n)
    seg += segment_size